In [46]:
import sys
import getpass
import os
import pandas as pd
import fiona
import geopandas as gpd
from arcgis import gis
username = getpass.getuser()

user = getpass.getuser()
sys.dont_write_bytecode = True

# for DataViz team members
sys.path.insert(0, '/Users/{}/Box/DataViz Projects/Utility Code'.format(user))

from utils_io import *

'aolsen@bayareametro.gov'

In [86]:
client = gis.GIS(url='https://arcgis.ad.mtc.ca.gov/portal/home/', username='{}@bayareametro.gov'.format(
    username) if not 'bayareametro' in username else username, verify_cert=False)

Enter password: ········


In [87]:
transit_dir = os.path.join('/Users',
                        username,
                        'Box',
                        'DataViz Projects',
                        'Spatial Analysis and Mapping',
                        'Active Transportation Plan',
                        'Data',
                        'regional_transit')

In [88]:
active_trans_dir = os.path.join('/Users',
                        username,
                        'Box',
                        'DataViz Projects',
                        'Spatial Analysis and Mapping',
                        'Active Transportation Plan',
                        'Data',
                        'Active Transportation.gdb')

## Read most recent transit data from

In [89]:
transit_rts_precovid = pull_geodata_from_argis(arcgis_data_id='2a519083d0a44a33940e469e427c8457',
                                               client=client)

In [90]:
transit_rts_current = gpd.read_file(active_trans_dir,
                            driver='FileGDB',
                            layer='regional_transit_routes_oct_2021')


In [91]:
#Rename current columns to match precovid columns
rename_dict = {'route_short_name':'route_s_nm',
               'route_long_name':'route_l_nm',
               'route_type_text':'route_type'}
#Drop preexisting route_type which is numerical. We don't want to have duplicate column names
transit_rts_current = transit_rts_current.drop(['route_type'],axis=1)
transit_rts_current.rename(columns=rename_dict,inplace=True)

In [92]:
transit_rts_current.rename_geometry(col='geom',inplace=True)

In [93]:
#Add col to pre-covid routes to indicate vintage
transit_rts_precovid['vintage'] = 'January 2020'

In [94]:
#Add col to current routes to indicate vintage
transit_rts_current['vintage'] = 'October 2021'

In [95]:
#Drop Napa Valley Vine Transit pre-covid route E
transit_rts_precovid.drop(transit_rts_precovid[transit_rts_precovid['route_id'] == 'VN:E'].index,
                          inplace=True)

In [96]:
#Subset new Napa Valley Vine Transit Routes
out_cols = ['route_id',
            'agency_id',
            'route_s_nm',
            'route_l_nm',
            'route_type',
            'vintage',
            'geom']
trans_rts_current_vn = transit_rts_current[out_cols][transit_rts_current['route_id']
                                                     .isin(['VN:E','VN:S','VN:N','VN:W'])]

In [98]:
transit_rts_precov_nv = pd.concat([transit_rts_precovid[out_cols],trans_rts_current_vn[out_cols]],axis=0,
                                  ignore_index=True,
                                  sort=False)

## Read transit routes classification xlsx

In [ ]:
os.listdir(transit_dir)

In [ ]:
transit_class = pd.read_excel(os.path.join(transit_dir,'BA_Agency_Category_0821.xlsx'),
                              sheet_name=2)

In [ ]:
transit_class.info()

In [ ]:
transit_rts_current[transit_rts_current['agency_id'] == 'CE']

In [ ]:
#Fix BART route ids to match shape ids
ba_dict = {'BA:BG-N': 'BA:Beige',
 'BA:BG-S': 'BA:Beige',
 'BA:BL-N': 'BA:Blue-Wkd/Sat',
 'BA:BL-S': 'BA:Blue-Wkd/Sat',
 'BA:GN-N': 'BA:Green',
 'BA:GN-S': 'BA:Green',
 'BA:OR-N': 'BA:Orange',
 'BA:OR-S': 'BA:Orange',
 'BA:PR-N': 'BA:Purple',
 'BA:PR-S': 'BA:Purple',
 'BA:RD-N': 'BA:Red',
 'BA:RD-S': 'BA:Red',
 'BA:YL-N': 'BA:Yellow',
 'BA:YL-S': 'BA:Yellow'}

transit_class['route_id'].replace(ba_dict,inplace=True)

In [ ]:
transit_rts_class = pd.merge(transit_rts_precovid[['route_id','geom']],
                             transit_class,
                             how='outer',
                             on='route_id',
                             indicator=True)

## Write non-match files to csv

In [ ]:
transit_rts_class[transit_rts_class['_merge'] == 'right_only']['agency_id'].unique()

In [ ]:
transit_rts_class[transit_rts_class['_merge'] == 'both']

In [ ]:
class_out_cols = ['agency_id',
                  'agency',
                  'route_short_name',
                  'route_long_name',
                  'category',
                  'sub_category',
                  'avg_peak_freq (min)',
                  'avg_offpeak_freq (min)',
                  'notes/comments']
class_export_df = transit_rts_class[class_out_cols][transit_rts_class['_merge'] == 'right_only'].copy()

In [ ]:
agency_ids = ['CE', 'CT', 'CC', 'SM', 'SF', 'VN', 'SC']
shape_out_cols = ['agency_id',
                  'agency_nm',
                  'route_id',
                  'route_l_nm',
                  'route_s_nm',
                  'route_type']
shape_export_df = transit_rts[shape_out_cols][transit_rts['agency_id'].isin(agency_ids)].copy()

In [ ]:
with pd.ExcelWriter(os.path.join(transit_dir,'nonmatching_route_ids.xlsx')) as writer:
    class_export_df.to_excel(writer, sheet_name='Unmatched_Route_Ids',index=False)
    shape_export_df.to_excel(writer, sheet_name='Potential_Route_Id_Matches',index=False)